In [1]:
import numpy as np

# データをランダムネットワーク符号化する関数
def encode_data(data, redundancy=1.2):
    """
    入力:
        data: 元データ（2D配列: 各行がブロック）
        redundancy: 冗長率 (例: 1.2 = 20% 冗長データ追加)
    出力:
        encoded_blocks: 符号化データ
        G: 符号化係数行列
    """
    n_blocks, block_size = data.shape
    n_encoded = int(n_blocks * redundancy)
    
    # ランダム係数行列を生成
    G = np.random.randint(0, 256, size=(n_encoded, n_blocks))
    
    # 符号化データの生成
    encoded_blocks = (G @ data) % 256
    return encoded_blocks, G

# 元データを生成 (例: 10ブロック×8バイト)
n_blocks = 10
block_size = 8
data = np.random.randint(0, 256, size=(n_blocks, block_size))

# 符号化を実行
encoded_blocks, G = encode_data(data)

# 結果を表示
print("元データ:")
print(data)
print("\n符号化データ:")
print(encoded_blocks)
print("\n符号化係数行列:")
print(G)


元データ:
[[ 48  86   7  66 194  19 140 238]
 [135  92 137 188 124  21  27  56]
 [ 59  92  33 239 184  18 187 215]
 [158  26 221 172  65 141 194 188]
 [  4  50 114  24  17  44 214 250]
 [118 236  21  96  41 161 211  53]
 [238 251  72 122 116  64 148  61]
 [249 105  85  80  92 189 230  42]
 [ 58  86 241 170  33  19 217 100]
 [132  54  99  71  10 136 213 243]]

符号化データ:
[[ 35 141 120 142  30  76 125   3]
 [224 179 244 137  31  73  76  40]
 [152  42   7  85 128 222  23 253]
 [129 111 180 147  90  77  11  65]
 [143 174 240 201 210  77 170 210]
 [ 56 139 228  42  15 118 101 167]
 [ 38  62 206 177  26  99 220 227]
 [153   3  15  71 252 248 133  21]
 [109  93 162  51 186 165 255  40]
 [174  92 157  95  72 240  18  56]
 [ 93 233  39 123 182 192 124 178]
 [203 214  51  35 171   6  88 141]]

符号化係数行列:
[[ 81 126  12 173 186  27 240 185 142  24]
 [ 99  60 213 254  74 203  74  95 234 184]
 [241 248  11  78 248 115 123  13  41 242]
 [ 90  48  92 118 194 108  48 197 244  93]
 [182   6 216  31  88 148  11 2

In [2]:
import numpy as np
import threading
import time
import random

# 符号化データ生成 (前ステップの関数を使用)
def encode_data(data, redundancy=1.2):
    n_blocks, block_size = data.shape
    n_encoded = int(n_blocks * redundancy)
    G = np.random.randint(0, 256, size=(n_encoded, n_blocks))
    encoded_blocks = (G @ data) % 256
    return encoded_blocks, G

# ノードのクラス
class PeerNode(threading.Thread):
    def __init__(self, node_id, total_nodes, data_size):
        super().__init__()
        self.node_id = node_id
        self.total_nodes = total_nodes
        self.data_size = data_size
        self.received_blocks = []
        self.coefficients = []

    def send_data(self, target_node, block, coefficient):
        """他のノードにデータを送信"""
        target_node.receive_data(block, coefficient)

    def receive_data(self, block, coefficient):
        """データを受信して保存"""
        self.received_blocks.append(block)
        self.coefficients.append(coefficient)

    def run(self):
        """データ送受信をシミュレート"""
        for _ in range(5):  # ランダムに5回送信
            target_node = random.choice(nodes)
            if target_node != self:
                block_index = random.randint(0, len(encoded_blocks) - 1)
                self.send_data(target_node, encoded_blocks[block_index], G[block_index])
            time.sleep(random.uniform(0.1, 0.5))  # ランダムな遅延をシミュレート

# 復号化関数
def decode_data(received_blocks, coefficients):
    try:
        # 行列のサイズを確認
        if len(received_blocks) >= len(coefficients[0]):
            C = np.array(received_blocks[:len(coefficients[0])])
            G_received = np.array(coefficients[:len(coefficients[0])])
            decoded_data = np.linalg.solve(G_received, C) % 256
            return decoded_data.astype(int)
        else:
            print("十分なブロックを受信していません。")
            return None
    except np.linalg.LinAlgError:
        print("復号化に失敗しました。線形独立でない可能性があります。")
        return None

# 元データを生成 (例: 10ブロック×8バイト)
n_blocks = 10
block_size = 8
data = np.random.randint(0, 256, size=(n_blocks, block_size))

# 符号化を実行
encoded_blocks, G = encode_data(data)

# ノードを初期化
num_nodes = 5
nodes = [PeerNode(node_id=i, total_nodes=num_nodes, data_size=(n_blocks, block_size)) for i in range(num_nodes)]

# ノードをスタート
for node in nodes:
    node.start()

# 全ノードがデータ送受信を終了するまで待つ
for node in nodes:
    node.join()

# 復号化の試行 (ノード0で行う)
decoded_data = decode_data(nodes[0].received_blocks, nodes[0].coefficients)
if decoded_data is not None:
    print("\n復号化されたデータ:")
    print(decoded_data)
    print("\n元データ:")
    print(data)
else:
    print("\nデータ復号化に失敗しました。")


十分なブロックを受信していません。

データ復号化に失敗しました。


In [3]:
import numpy as np
import threading
import time
import random

# 符号化データ生成 (前ステップの関数を使用)
def encode_data(data, redundancy=1.2):
    n_blocks, block_size = data.shape
    n_encoded = int(n_blocks * redundancy)
    G = np.random.randint(1, 256, size=(n_encoded, n_blocks))  # 係数を1以上に制限
    encoded_blocks = (G @ data) % 256
    return encoded_blocks, G

# ノードのクラス
class PeerNode(threading.Thread):
    def __init__(self, node_id, total_nodes, data_size):
        super().__init__()
        self.node_id = node_id
        self.total_nodes = total_nodes
        self.data_size = data_size
        self.received_blocks = []
        self.coefficients = []
        self.running = True

    def send_data(self, target_node, block, coefficient):
        """他のノードにデータを送信"""
        target_node.receive_data(block, coefficient)

    def receive_data(self, block, coefficient):
        """データを受信して保存"""
        self.received_blocks.append(block)
        self.coefficients.append(coefficient)

    def run(self):
        """データ送受信をシミュレート"""
        for _ in range(10):  # 各ノードが10回送信を試みる
            if not self.running:
                break
            target_node = random.choice(nodes)
            if target_node != self:
                block_index = random.randint(0, len(encoded_blocks) - 1)
                self.send_data(target_node, encoded_blocks[block_index], G[block_index])
            time.sleep(random.uniform(0.1, 0.5))  # ランダムな遅延をシミュレート
        self.running = False

# 復号化関数
def decode_data(received_blocks, coefficients):
    try:
        # 行列のサイズを確認
        if len(received_blocks) >= n_blocks:
            C = np.array(received_blocks[:n_blocks])
            G_received = np.array(coefficients[:n_blocks])
            
            # 線形独立性の確認
            if np.linalg.matrix_rank(G_received) == n_blocks:
                decoded_data = np.linalg.solve(G_received, C) % 256
                return decoded_data.astype(int)
            else:
                print("受信した行列が線形独立ではありません。")
                return None
        else:
            print("十分なブロックを受信していません。")
            return None
    except np.linalg.LinAlgError:
        print("復号化に失敗しました。")
        return None

# 元データを生成 (例: 10ブロック×8バイト)
n_blocks = 10
block_size = 8
data = np.random.randint(0, 256, size=(n_blocks, block_size))

# 符号化を実行
encoded_blocks, G = encode_data(data)

# ノードを初期化
num_nodes = 5
nodes = [PeerNode(node_id=i, total_nodes=num_nodes, data_size=(n_blocks, block_size)) for i in range(num_nodes)]

# ノードをスタート
for node in nodes:
    node.start()

# 全ノードがデータ送受信を終了するまで待つ
for node in nodes:
    node.join()

# 復号化の試行 (ノード0で行う)
decoded_data = decode_data(nodes[0].received_blocks, nodes[0].coefficients)
if decoded_data is not None:
    print("\n復号化されたデータ:")
    print(decoded_data)
    print("\n元データ:")
    print(data)
else:
    print("\nデータ復号化に失敗しました。")


受信した行列が線形独立ではありません。

データ復号化に失敗しました。


In [4]:
import numpy as np
import threading
import time
import random

# 符号化データ生成 (前ステップの関数を使用)
def encode_data(data, redundancy=1.2):
    n_blocks, block_size = data.shape
    n_encoded = int(n_blocks * redundancy)
    while True:
        G = np.random.randint(1, 256, size=(n_encoded, n_blocks))  # 係数を1以上に制限
        if np.linalg.matrix_rank(G) == n_blocks:  # 線形独立性を確認
            break
    encoded_blocks = (G @ data) % 256
    return encoded_blocks, G

# ノードのクラス
class PeerNode(threading.Thread):
    def __init__(self, node_id, total_nodes, data_size):
        super().__init__()
        self.node_id = node_id
        self.total_nodes = total_nodes
        self.data_size = data_size
        self.received_blocks = []
        self.coefficients = []
        self.running = True

    def send_data(self, target_node, block, coefficient):
        """他のノードにデータを送信"""
        target_node.receive_data(block, coefficient)

    def receive_data(self, block, coefficient):
        """データを受信して保存"""
        if len(self.coefficients) == 0:  # 初回は必ず受信
            self.received_blocks.append(block)
            self.coefficients.append(coefficient)
        else:
            # 新しい行が既存の行列と線形独立かを確認
            G_current = np.array(self.coefficients)
            new_G = np.vstack([G_current, coefficient])
            if np.linalg.matrix_rank(new_G) > len(G_current):  # ランクが増えるなら追加
                self.received_blocks.append(block)
                self.coefficients.append(coefficient)

    def run(self):
        """データ送受信をシミュレート"""
        for _ in range(10):  # 各ノードが10回送信を試みる
            if not self.running:
                break
            target_node = random.choice(nodes)
            if target_node != self:
                block_index = random.randint(0, len(encoded_blocks) - 1)
                self.send_data(target_node, encoded_blocks[block_index], G[block_index])
            time.sleep(random.uniform(0.1, 0.5))  # ランダムな遅延をシミュレート
        self.running = False

# 復号化関数
def decode_data(received_blocks, coefficients):
    try:
        # 行列のサイズを確認
        if len(received_blocks) >= n_blocks:
            C = np.array(received_blocks[:n_blocks])
            G_received = np.array(coefficients[:n_blocks])
            
            # 線形独立性の確認
            if np.linalg.matrix_rank(G_received) == n_blocks:
                decoded_data = np.linalg.solve(G_received, C) % 256
                return decoded_data.astype(int)
            else:
                print("受信した行列が線形独立ではありません。")
                return None
        else:
            print("十分なブロックを受信していません。")
            return None
    except np.linalg.LinAlgError:
        print("復号化に失敗しました。")
        return None

# 元データを生成 (例: 10ブロック×8バイト)
n_blocks = 10
block_size = 8
data = np.random.randint(0, 256, size=(n_blocks, block_size))

# 符号化を実行
encoded_blocks, G = encode_data(data)

# ノードを初期化
num_nodes = 5
nodes = [PeerNode(node_id=i, total_nodes=num_nodes, data_size=(n_blocks, block_size)) for i in range(num_nodes)]

# ノードをスタート
for node in nodes:
    node.start()

# 全ノードがデータ送受信を終了するまで待つ
for node in nodes:
    node.join()

# 復号化の試行 (ノード0で行う)
decoded_data = decode_data(nodes[0].received_blocks, nodes[0].coefficients)
if decoded_data is not None:
    print("\n復号化されたデータ:")
    print(decoded_data)
    print("\n元データ:")
    print(data)
else:
    print("\nデータ復号化に失敗しました。")


十分なブロックを受信していません。

データ復号化に失敗しました。


In [5]:
import numpy as np
import threading
import time
import random

# 符号化データ生成 (前ステップの関数を使用)
def encode_data(data, redundancy=1.5):
    n_blocks, block_size = data.shape
    n_encoded = int(n_blocks * redundancy)
    while True:
        G = np.random.randint(1, 256, size=(n_encoded, n_blocks))
        if np.linalg.matrix_rank(G) == n_blocks:  # 線形独立性を確認
            break
    encoded_blocks = (G @ data) % 256
    return encoded_blocks, G

# ノードのクラス
class PeerNode(threading.Thread):
    def __init__(self, node_id, total_nodes):
        super().__init__()
        self.node_id = node_id
        self.total_nodes = total_nodes
        self.received_blocks = []
        self.coefficients = []
        self.running = True

    def send_data(self, target_node, block, coefficient):
        """他のノードにデータを送信"""
        target_node.receive_data(block, coefficient)

    def receive_data(self, block, coefficient):
        """データを受信して保存"""
        if len(self.coefficients) == 0:  # 初回は必ず受信
            self.received_blocks.append(block)
            self.coefficients.append(coefficient)
        else:
            # 新しい行が既存の行列と線形独立かを確認
            G_current = np.array(self.coefficients)
            new_G = np.vstack([G_current, coefficient])
            if np.linalg.matrix_rank(new_G) > len(G_current):  # ランクが増えるなら追加
                self.received_blocks.append(block)
                self.coefficients.append(coefficient)

    def run(self):
        """データ送受信をシミュレート"""
        for _ in range(20):  # 各ノードが20回送信を試みる
            if not self.running:
                break
            target_node = random.choice(nodes)
            if target_node != self:
                block_index = random.randint(0, len(encoded_blocks) - 1)
                self.send_data(target_node, encoded_blocks[block_index], G[block_index])
            time.sleep(random.uniform(0.1, 0.2))  # ランダムな遅延をシミュレート
        self.running = False

# 復号化関数
def decode_data(received_blocks, coefficients):
    try:
        if len(received_blocks) >= n_blocks:
            C = np.array(received_blocks[:n_blocks])
            G_received = np.array(coefficients[:n_blocks])
            
            # 線形独立性の確認
            if np.linalg.matrix_rank(G_received) == n_blocks:
                decoded_data = np.linalg.solve(G_received, C) % 256
                return decoded_data.astype(int)
            else:
                print("受信した行列が線形独立ではありません。")
                return None
        else:
            print(f"十分なブロックを受信していません。現在の受信数: {len(received_blocks)}")
            return None
    except np.linalg.LinAlgError:
        print("復号化に失敗しました。")
        return None

# 元データを生成 (例: 10ブロック×8バイト)
n_blocks = 10
block_size = 8
data = np.random.randint(0, 256, size=(n_blocks, block_size))

# 符号化を実行
encoded_blocks, G = encode_data(data)

# ノードを初期化
num_nodes = 5
nodes = [PeerNode(node_id=i, total_nodes=num_nodes) for i in range(num_nodes)]

# ノードをスタート
for node in nodes:
    node.start()

# 全ノードがデータ送受信を終了するまで待つ
for node in nodes:
    node.join()

# 各ノードの受信データ数を確認
for i, node in enumerate(nodes):
    print(f"ノード {i} が受信したブロック数: {len(node.received_blocks)}")

# 復号化の試行 (ノード0で行う)
decoded_data = decode_data(nodes[0].received_blocks, nodes[0].coefficients)
if decoded_data is not None:
    print("\n復号化されたデータ:")
    print(decoded_data)
    print("\n元データ:")
    print(data)
else:
    print("\nデータ復号化に失敗しました。")


ノード 0 が受信したブロック数: 8
ノード 1 が受信したブロック数: 10
ノード 2 が受信したブロック数: 10
ノード 3 が受信したブロック数: 9
ノード 4 が受信したブロック数: 9
十分なブロックを受信していません。現在の受信数: 8

データ復号化に失敗しました。


In [6]:
import numpy as np
import threading
import time
import random

# 符号化データ生成
def encode_data(data, redundancy=1.5):
    n_blocks, block_size = data.shape
    n_encoded = int(n_blocks * redundancy)
    while True:
        G = np.random.randint(1, 256, size=(n_encoded, n_blocks))
        if np.linalg.matrix_rank(G) == n_blocks:  # 線形独立性を確認
            break
    encoded_blocks = (G @ data) % 256
    return encoded_blocks, G

# ノードのクラス
class PeerNode(threading.Thread):
    def __init__(self, node_id, total_nodes):
        super().__init__()
        self.node_id = node_id
        self.total_nodes = total_nodes
        self.received_blocks = []
        self.coefficients = []
        self.running = True

    def send_data(self, target_node, block, coefficient):
        """他のノードにデータを送信"""
        target_node.receive_data(block, coefficient)

    def receive_data(self, block, coefficient):
        """データを受信して保存"""
        if len(self.coefficients) == 0:  # 初回は必ず受信
            self.received_blocks.append(block)
            self.coefficients.append(coefficient)
        else:
            # 新しい行が既存の行列と線形独立かを確認
            G_current = np.array(self.coefficients)
            new_G = np.vstack([G_current, coefficient])
            if np.linalg.matrix_rank(new_G) > len(G_current):  # ランクが増えるなら追加
                self.received_blocks.append(block)
                self.coefficients.append(coefficient)

    def run(self):
        """データ送受信をシミュレート"""
        while self.running:
            target_node = random.choice(nodes)
            if target_node != self:
                block_index = random.randint(0, len(encoded_blocks) - 1)
                self.send_data(target_node, encoded_blocks[block_index], G[block_index])
            time.sleep(random.uniform(0.1, 0.2))  # ランダムな遅延をシミュレート

# 復号化関数
def decode_data(received_blocks, coefficients):
    try:
        if len(received_blocks) >= n_blocks:
            C = np.array(received_blocks[:n_blocks])
            G_received = np.array(coefficients[:n_blocks])
            
            if np.linalg.matrix_rank(G_received) == n_blocks:
                decoded_data = np.linalg.solve(G_received, C) % 256
                return decoded_data.astype(int)
            else:
                print("受信した行列が線形独立ではありません。")
                return None
        else:
            print(f"十分なブロックを受信していません。現在の受信数: {len(received_blocks)}")
            return None
    except np.linalg.LinAlgError:
        print("復号化に失敗しました。")
        return None

# 元データを生成 (例: 10ブロック×8バイト)
n_blocks = 10
block_size = 8
data = np.random.randint(0, 256, size=(n_blocks, block_size))

# 符号化を実行
encoded_blocks, G = encode_data(data)

# ノードを初期化
num_nodes = 5
nodes = [PeerNode(node_id=i, total_nodes=num_nodes) for i in range(num_nodes)]

# ノードをスタート
for node in nodes:
    node.start()

# データ伝播が完了するまで監視
while True:
    time.sleep(2)  # 進捗確認の間隔
    received_counts = [len(node.received_blocks) for node in nodes]
    print(f"現在の受信ブロック数: {received_counts}")
    
    if all(count >= n_blocks for count in received_counts):
        print("全ノードが十分なブロックを受信しました。")
        for node in nodes:
            node.running = False
        break

# 全ノードが停止するのを待つ
for node in nodes:
    node.join()

# 復号化の試行 (ノード0で行う)
decoded_data = decode_data(nodes[0].received_blocks, nodes[0].coefficients)
if decoded_data is not None:
    print("\n復号化されたデータ:")
    print(decoded_data)
    print("\n元データ:")
    print(data)
else:
    print("\nデータ復号化に失敗しました。")


現在の受信ブロック数: [8, 8, 7, 5, 7]
現在の受信ブロック数: [10, 10, 10, 10, 10]
全ノードが十分なブロックを受信しました。

復号化されたデータ:
[[243  18 240 250  14 243   5  11]
 [253   4 253 255   3 253   0   2]
 [  2 252   3   1 251   3 255 254]
 [  3 249   5   1 251   4 253 252]
 [  4 251   4   2 252   2 255 251]
 [  0   1 255   0   1 255   0   0]
 [255   0 255 255   0 254 255   1]
 [253   4 252 254   2 252   0   3]
 [ 15 232  19   7 237  17 250 241]
 [  3 249   5   1 251   4 255 252]]

元データ:
[[244  28  19   2  51 178 109 154]
 [238  96 187 129  22 200 201  64]
 [171  44 252 103   9  43  18 183]
 [165 157 254 240 107  66 105   0]
 [ 15 223 249 158   1   1 117  31]
 [ 74 158 128 158  22  34 167   0]
 [131 238 244 122  55 221 148  41]
 [243 106 109  85 160 127 109  13]
 [202 129 222 194  45 135 197 140]
 [108 254 248 231 208  43  41  78]]


In [7]:
from pyfinite import ffield

# 有限体 GF(256) を初期化
F = ffield.FField(8)

def gf256_add(x, y):
    """GF(256) での加算"""
    return x ^ y

def gf256_mul(x, y):
    """GF(256) での乗算"""
    return F.Multiply(x, y)

def gf256_div(x, y):
    """GF(256) での除算"""
    return F.Multiply(x, F.Inverse(y))

def gf256_matrix_inverse(matrix):
    """GF(256) 上の行列逆行列"""
    size = len(matrix)
    I = np.eye(size, dtype=int)  # 単位行列
    M = np.array(matrix, dtype=int)

    for col in range(size):
        # 対角成分を1にする
        diag_element = M[col, col]
        if diag_element == 0:
            raise ValueError("行列が非正則です。")
        inv_diag = F.Inverse(diag_element)
        M[col, :] = [gf256_mul(val, inv_diag) for val in M[col, :]]
        I[col, :] = [gf256_mul(val, inv_diag) for val in I[col, :]]

        # 他の行の該当列をゼロにする
        for row in range(size):
            if row != col:
                factor = M[row, col]
                M[row, :] = [gf256_add(M[row, k], gf256_mul(factor, M[col, k])) for k in range(size)]
                I[row, :] = [gf256_add(I[row, k], gf256_mul(factor, I[col, k])) for k in range(size)]

    return I

def decode_data_gf256(received_blocks, coefficients):
    """有限体 GF(256) でデータを復号化"""
    try:
        if len(received_blocks) >= n_blocks:
            C = np.array(received_blocks[:n_blocks], dtype=int)
            G_received = np.array(coefficients[:n_blocks], dtype=int)

            # 行列の逆行列を求めて復号化
            G_inverse = gf256_matrix_inverse(G_received)
            decoded_data = np.dot(G_inverse, C) % 256
            return decoded_data.astype(int)
        else:
            print(f"十分なブロックを受信していません。現在の受信数: {len(received_blocks)}")
            return None
    except ValueError as e:
        print(f"復号化に失敗しました: {e}")
        return None


In [8]:
# デバッグ: 復号化前の情報を表示
print("\nデバッグ: 受信したブロック数")
print([len(node.received_blocks) for node in nodes])

print("\nデバッグ: ノード 0 の受信データと係数行列")
print("受信データ:", nodes[0].received_blocks)
print("係数行列:", nodes[0].coefficients)

# 復号化を試行
decoded_data = decode_data_gf256(nodes[0].received_blocks, nodes[0].coefficients)
if decoded_data is not None:
    print("\n復号化されたデータ:")
    print(decoded_data)
    print("\n元データ:")
    print(data)
else:
    print("\nデータ復号化に失敗しました。")



デバッグ: 受信したブロック数
[10, 10, 10, 10, 10]

デバッグ: ノード 0 の受信データと係数行列
受信データ: [array([200, 114, 186, 233, 171, 213, 225,   6]), array([ 30, 141,  84, 231, 176,  23,  42, 186]), array([ 94, 216, 122, 203, 127,  94, 221, 104]), array([243, 165, 133, 232, 172,  96, 110, 115]), array([165,  67, 224, 165, 163, 239,   7,  96]), array([231,  25, 247, 226,  11, 188,  41, 207]), array([183, 176, 144, 112, 204,  96,  17, 232]), array([245, 116, 215,  88,  21, 224, 181,  25]), array([ 26,  36, 102,  90, 168, 155,  87, 178]), array([ 45, 189,  49,  86,  85, 217, 221, 252])]
係数行列: [array([136, 155, 112,   4, 111, 164, 117,  52,  71, 160]), array([182, 167, 207, 238, 171,  77, 192,  46,   2, 199]), array([200,  61, 201,  12, 128, 219, 254,   3, 112,  38]), array([121,  97, 212, 196, 118, 230,  28,  95,  17,  36]), array([225, 216,  59,  51,  79, 107, 134, 234, 213,  54]), array([203, 116, 242, 184, 132, 239, 255, 108,  69, 140]), array([224, 168, 127, 181, 158, 146, 169, 254, 105, 203]), array([251,  35, 20

In [9]:
import numpy as np
from pyfinite import ffield

# GF(256) の有限体の設定
F = ffield.FField(8)

# 符号化行列（例）
coefficients = np.array([
    [197, 253, 106, 148, 99, 196, 157, 58, 30, 76],
    [108, 18, 208, 193, 162, 133, 7, 24, 89, 65],
    [174, 254, 55, 223, 224, 132, 72, 167, 81, 104],
    [132, 45, 137, 21, 72, 141, 96, 186, 3, 217],
    [27, 202, 189, 48, 137, 44, 27, 248, 83, 197],
    [68, 204, 205, 84, 63, 178, 127, 52, 30, 14],
    [130, 82, 15, 239, 97, 150, 8, 225, 128, 42],
    [76, 69, 44, 150, 216, 121, 17, 56, 12, 161],
    [22, 70, 37, 118, 33, 216, 232, 228, 138, 154],
    [236, 2, 108, 2, 26, 43, 172, 60, 229, 163]
])

n_blocks = len(coefficients)  # データブロックの数

# 係数行列のランクを確認
G_received = np.array(coefficients[:n_blocks], dtype=int)
rank = np.linalg.matrix_rank(G_received)
print(f"係数行列のランク: {rank}/{n_blocks}")
if rank < n_blocks:
    print("符号化行列がフルランクではありません。復号化に失敗する可能性があります。")
else:
    print("符号化行列はフルランクです。")


係数行列のランク: 10/10
符号化行列はフルランクです。


In [10]:
import numpy as np
from pyfinite import ffield

# GF(256) の有限体の設定
F = ffield.FField(8)

# 符号化行列の作成 (例)
coefficients = np.array([
    [197, 253, 106, 148, 99, 196, 157, 58, 30, 76],
    [108, 18, 208, 193, 162, 133, 7, 24, 89, 65],
    [174, 254, 55, 223, 224, 132, 72, 167, 81, 104],
    [132, 45, 137, 21, 72, 141, 96, 186, 3, 217],
    [27, 202, 189, 48, 137, 44, 27, 248, 83, 197],
    [68, 204, 205, 84, 63, 178, 127, 52, 30, 14],
    [130, 82, 15, 239, 97, 150, 8, 225, 128, 42],
    [76, 69, 44, 150, 216, 121, 17, 56, 12, 161],
    [22, 70, 37, 118, 33, 216, 232, 228, 138, 154],
    [236, 2, 108, 2, 26, 43, 172, 60, 229, 163]
])

# 入力データの作成 (例)
data = np.array([
    [131, 28, 100, 60, 65, 237, 141, 41],
    [133, 134, 40, 173, 174, 249, 80, 157],
    [163, 162, 19, 7, 200, 35, 133, 82],
    [180, 193, 231, 219, 30, 16, 49, 46],
    [195, 143, 169, 104, 180, 26, 36, 229],
    [158, 114, 33, 56, 94, 97, 114, 64],
    [23, 123, 195, 108, 65, 135, 104, 187],
    [64, 10, 51, 120, 222, 34, 16, 194],
    [219, 36, 67, 81, 232, 128, 102, 155],
    [108, 147, 241, 131, 51, 162, 255, 42]
])

# 行列の演算をGF(256)で行う
def gf256_matrix_multiply(A, B, F):
    result = []
    for A_row in A:
        row_result = []
        for B_col in B.T:
            sum_result = 0
            for a, b in zip(A_row, B_col):
                sum_result = F.Add(sum_result, F.Multiply(a, b))  # AddとMultiplyを使って加算と乗算を行う
            row_result.append(sum_result)
        result.append(row_result)
    return np.array(result, dtype=int)

# 符号化行列の演算例
encoded_data = gf256_matrix_multiply(coefficients, data, F)
print("符号化後のデータ:")
print(encoded_data)


符号化後のデータ:
[[105 106 135 171  93  91 236 212]
 [241 173 105 204 181  40  42  45]
 [ 86 209  43  72 117  86  13  98]
 [ 79 173 178  15  70 147 116   9]
 [182  49 197  88 248 109 187   9]
 [214  16  92 244  64  36  67 236]
 [183 227 246 113 242 214 229 217]
 [ 14 164 243   4  56 242  49  77]
 [162 235 125  35 188 105  28  54]
 [ 69  65 145 142 178 127 144  53]]


In [11]:
import numpy as np
from pyfinite import ffield

# GF(256) の有限体の設定
F = ffield.FField(8)

# 符号化行列の作成 (例)
coefficients = np.array([
    [197, 253, 106, 148, 99, 196, 157, 58, 30, 76],
    [108, 18, 208, 193, 162, 133, 7, 24, 89, 65],
    [174, 254, 55, 223, 224, 132, 72, 167, 81, 104],
    [132, 45, 137, 21, 72, 141, 96, 186, 3, 217],
    [27, 202, 189, 48, 137, 44, 27, 248, 83, 197],
    [68, 204, 205, 84, 63, 178, 127, 52, 30, 14],
    [130, 82, 15, 239, 97, 150, 8, 225, 128, 42],
    [76, 69, 44, 150, 216, 121, 17, 56, 12, 161],
    [22, 70, 37, 118, 33, 216, 232, 228, 138, 154],
    [236, 2, 108, 2, 26, 43, 172, 60, 229, 163]
])

# 入力データの作成 (符号化されたデータ)
encoded_data = np.array([
    [105, 106, 135, 171, 93, 91, 236, 212],
    [241, 173, 105, 204, 181, 40, 42, 45],
    [86, 209, 43, 72, 117, 86, 13, 98],
    [79, 173, 178, 15, 70, 147, 116, 9],
    [182, 49, 197, 88, 248, 109, 187, 9],
    [214, 16, 92, 244, 64, 36, 67, 236],
    [183, 227, 246, 113, 242, 214, 229, 217],
    [14, 164, 243, 4, 56, 242, 49, 77],
    [162, 235, 125, 35, 188, 105, 28, 54],
    [69, 65, 145, 142, 178, 127, 144, 53]
])

# 行列の演算をGF(256)で行う
def gf256_matrix_multiply(A, B, F):
    result = []
    for A_row in A:
        row_result = []
        for B_col in B.T:
            sum_result = 0
            for a, b in zip(A_row, B_col):
                sum_result = F.Add(sum_result, F.Multiply(a, b))  # AddとMultiplyを使って加算と乗算を行う
            row_result.append(sum_result)
        result.append(row_result)
    return np.array(result, dtype=int)

# 符号化行列の逆行列を計算
def gf256_matrix_inverse(A, F):
    n = A.shape[0]
    augmented_matrix = np.hstack([A, np.eye(n, dtype=int)])
    for i in range(n):
        if augmented_matrix[i, i] == 0:
            for j in range(i + 1, n):
                if augmented_matrix[j, i] != 0:
                    augmented_matrix[[i, j]] = augmented_matrix[[j, i]]
                    break
        for j in range(i + 1, n):
            if augmented_matrix[j, i] != 0:
                factor = F.Inverse(augmented_matrix[i, i])
                factor = F.Multiply(factor, augmented_matrix[j, i])
                for k in range(2 * n):  # 行の要素ごとに処理
                    augmented_matrix[j, k] = F.Add(augmented_matrix[j, k], F.Multiply(factor, augmented_matrix[i, k]))
    for i in range(n - 1, -1, -1):
        for j in range(i - 1, -1, -1):
            if augmented_matrix[j, i] != 0:
                factor = F.Inverse(augmented_matrix[i, i])
                factor = F.Multiply(factor, augmented_matrix[j, i])
                for k in range(2 * n):  # 行の要素ごとに処理
                    augmented_matrix[j, k] = F.Add(augmented_matrix[j, k], F.Multiply(factor, augmented_matrix[i, k]))
    return augmented_matrix[:, n:]

# 復号化
def decode(encoded_data, coefficients, F):
    # 符号化行列の逆行列を計算
    inv_coefficients = gf256_matrix_inverse(coefficients, F)
    
    # 復号化
    decoded_data = gf256_matrix_multiply(inv_coefficients, encoded_data, F)
    return decoded_data

# 復号化実行
decoded_data = decode(encoded_data, coefficients, F)

print("復号化されたデータ:")
print(decoded_data)


復号化されたデータ:
[[124 153 161  28 210  44 190  38]
 [ 82 147 202 152  89 251 137  20]
 [145  24 254 152  38 118 112 138]
 [112 144  29  13   8 110 208 186]
 [194  46  88 242 102 178  30 180]
 [124 109 248  75  70 238 109  22]
 [189  73 200 244 179  27 127 161]
 [179 195 156 125 144 232 229 247]
 [237 145 148  82  83 112  97 213]
 [164  45 202 154 143 208 137 212]]
